In [1]:
import pandas as pd

# Load the uploaded CSV files
projections_path = "C:/Users/user/Desktop/학부 연구생/프로젝트/archive/indiana_projections.csv"
reports_path ="C:/Users/user/Desktop/학부 연구생/프로젝트/archive/indiana_reports.csv"


projections_df = pd.read_csv(projections_path)
reports_df = pd.read_csv(reports_path)

# 데이터 구조 확인
projections_df_info = projections_df.info()
reports_df_info = reports_df.info()

projections_head = projections_df.head()
reports_head = reports_df.head()

(projections_df_info, reports_df_info, projections_head, reports_head)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7466 entries, 0 to 7465
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         7466 non-null   int64 
 1   filename    7466 non-null   object
 2   projection  7466 non-null   object
dtypes: int64(1), object(2)
memory usage: 175.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3851 entries, 0 to 3850
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         3851 non-null   int64 
 1   MeSH        3851 non-null   object
 2   Problems    3851 non-null   object
 3   image       3851 non-null   object
 4   indication  3765 non-null   object
 5   comparison  2685 non-null   object
 6   findings    3337 non-null   object
 7   impression  3820 non-null   object
dtypes: int64(1), object(7)
memory usage: 240.8+ KB


(None,
 None,
    uid                filename projection
 0    1  1_IM-0001-4001.dcm.png    Frontal
 1    1  1_IM-0001-3001.dcm.png    Lateral
 2    2  2_IM-0652-1001.dcm.png    Frontal
 3    2  2_IM-0652-2001.dcm.png    Lateral
 4    3  3_IM-1384-1001.dcm.png    Frontal,
    uid                                               MeSH  \
 0    1                                             normal   
 1    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
 2    3                                             normal   
 3    4  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4    5  Osteophyte/thoracic vertebrae/multiple/small;T...   
 
                                             Problems  \
 0                                             normal   
 1                      Cardiomegaly;Pulmonary Artery   
 2                                             normal   
 3  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4                         Osteophyte;Thickening;Lung   
 
         

In [2]:
# Filter for Frontal images
frontal_projections = projections_df[projections_df['projection'] == 'Frontal']

# Merge with reports based on 'uid'
merged_df = pd.merge(frontal_projections, reports_df, on='uid')

# Select 'Problems' instead of 'MeSH' for labels
filtered_data_with_problems = merged_df[['filename', 'Problems']]

# Check the distribution of 'Problems'
filtered_data_with_problems.head(), filtered_data_with_problems['Problems'].str.split(';').explode().value_counts()


(                    filename  \
 0     1_IM-0001-4001.dcm.png   
 1     2_IM-0652-1001.dcm.png   
 2     3_IM-1384-1001.dcm.png   
 3     4_IM-2050-1001.dcm.png   
 4  5_IM-2117-1003002.dcm.png   
 
                                             Problems  
 0                                             normal  
 1                      Cardiomegaly;Pulmonary Artery  
 2                                             normal  
 3  Pulmonary Disease, Chronic Obstructive;Bullous...  
 4                         Osteophyte;Thickening;Lung  ,
 Problems
 normal                   1387
 Lung                      545
 Opacity                   501
 Calcinosis                332
 Cardiomegaly              331
                          ... 
 Cholelithiasis              1
 Colonic Interposition       1
 Fibrosis                    1
 Trachea                     1
 Hemothorax                  1
 Name: count, Length: 119, dtype: int64)

In [3]:
# Extract the first label from the 'Problems' field
filtered_data_with_problems['Primary_Label'] = filtered_data_with_problems['Problems']
# Check the label distribution for the primary label
filtered_data_with_primary_label = filtered_data_with_problems[['filename', 'Primary_Label']]
label_distribution = filtered_data_with_primary_label['Primary_Label'].value_counts()

# Display the updated dataset and the label distribution
filtered_data_with_primary_label.head(), label_distribution


C:\Users\user\AppData\Local\Temp\ipykernel_16368\678683256.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_with_problems['Primary_Label'] = filtered_data_with_problems['Problems']


(                    filename  \
 0     1_IM-0001-4001.dcm.png   
 1     2_IM-0652-1001.dcm.png   
 2     3_IM-1384-1001.dcm.png   
 3     4_IM-2050-1001.dcm.png   
 4  5_IM-2117-1003002.dcm.png   
 
                                        Primary_Label  
 0                                             normal  
 1                      Cardiomegaly;Pulmonary Artery  
 2                                             normal  
 3  Pulmonary Disease, Chronic Obstructive;Bullous...  
 4                         Osteophyte;Thickening;Lung  ,
 Primary_Label
 normal                                                                                                                             1387
 No Indexing                                                                                                                          91
 Lung                                                                                                                                 86
 Calcified Granuloma                 

In [4]:
# Prepare filenames and labels for single-label classification
filenames_single_label = filtered_data_with_primary_label['filename'].values
labels_single_label = filtered_data_with_primary_label['Primary_Label'].values

# Encode labels to integers using sklearn's LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels_single_label)

# Map encoded labels back to their original names for reference
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# Split data into train, validation, and test sets
X_train_filenames, X_temp_filenames, y_train_labels, y_temp_labels = train_test_split(
    filenames_single_label, encoded_labels, test_size=0.3, random_state=42
)
X_val_filenames, X_test_filenames, y_val_labels, y_test_labels = train_test_split(
    X_temp_filenames, y_temp_labels, test_size=0.5, random_state=42
)

# Display dataset information
len(X_train_filenames), len(X_val_filenames), len(X_test_filenames), label_mapping


(2672,
 573,
 573,
 {'Airspace Disease': 0,
  'Airspace Disease;Calcified Granuloma;Catheters, Indwelling': 1,
  'Airspace Disease;Consolidation;Catheters, Indwelling;Pneumonia': 2,
  'Airspace Disease;Costophrenic Angle;Catheters, Indwelling;Pleural Effusion': 3,
  'Airspace Disease;Costophrenic Angle;Cicatrix': 4,
  'Airspace Disease;Diaphragm': 5,
  'Airspace Disease;Opacity;Airspace Disease;Pneumonia': 6,
  'Airspace Disease;Opacity;Deformity': 7,
  'Airspace Disease;Opacity;Pneumonia;Nodule': 8,
  'Airspace Disease;Opacity;Pulmonary Atelectasis;Deformity': 9,
  'Airspace Disease;Pleural Effusion;Calcinosis;Infiltrate;Pulmonary Atelectasis': 10,
  'Airspace Disease;Pleural Effusion;Pneumothorax;Pulmonary Atelectasis;Medical Device': 11,
  'Airspace Disease;Pleural Effusion;Spine': 12,
  'Airspace Disease;Pneumonia': 13,
  'Airspace Disease;Pulmonary Atelectasis;Consolidation;Cardiomegaly;Tube, Inserted': 14,
  'Airspace Disease;Pulmonary Atelectasis;Infiltrate': 15,
  'Airspace Dis

In [5]:
from sklearn.model_selection import train_test_split

# Split data into train, validation, and test sets
X_train_filenames, X_temp_filenames, y_train_labels, y_temp_labels = train_test_split(
    filenames_single_label, encoded_labels, test_size=0.3, random_state=42
)
X_val_filenames, X_test_filenames, y_val_labels, y_test_labels = train_test_split(
    X_temp_filenames, y_temp_labels, test_size=0.5, random_state=42
)

# Display dataset information: train, validation, test sizes and label mapping
len(X_train_filenames), len(X_val_filenames), len(X_test_filenames), label_mapping


(2672,
 573,
 573,
 {'Airspace Disease': 0,
  'Airspace Disease;Calcified Granuloma;Catheters, Indwelling': 1,
  'Airspace Disease;Consolidation;Catheters, Indwelling;Pneumonia': 2,
  'Airspace Disease;Costophrenic Angle;Catheters, Indwelling;Pleural Effusion': 3,
  'Airspace Disease;Costophrenic Angle;Cicatrix': 4,
  'Airspace Disease;Diaphragm': 5,
  'Airspace Disease;Opacity;Airspace Disease;Pneumonia': 6,
  'Airspace Disease;Opacity;Deformity': 7,
  'Airspace Disease;Opacity;Pneumonia;Nodule': 8,
  'Airspace Disease;Opacity;Pulmonary Atelectasis;Deformity': 9,
  'Airspace Disease;Pleural Effusion;Calcinosis;Infiltrate;Pulmonary Atelectasis': 10,
  'Airspace Disease;Pleural Effusion;Pneumothorax;Pulmonary Atelectasis;Medical Device': 11,
  'Airspace Disease;Pleural Effusion;Spine': 12,
  'Airspace Disease;Pneumonia': 13,
  'Airspace Disease;Pulmonary Atelectasis;Consolidation;Cardiomegaly;Tube, Inserted': 14,
  'Airspace Disease;Pulmonary Atelectasis;Infiltrate': 15,
  'Airspace Dis

In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

# 이미지 디렉토리 경로
image_dir = r'C:/Users/user/Desktop/images_normalized'

# 이미지 데이터와 레이블을 저장할 리스트
# Load and preprocess images for single-label classification
def load_images_from_filenames(filenames, directory, target_size):
    images = []
    for filename in filenames:
        img_path = os.path.join(directory, filename)
        try:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
            images.append(np.zeros((*target_size, 3)))
    return np.array(images)

# Image parameters
img_size = (224, 224)
    
# Load and preprocess images for train, validation, and test sets
X_train_images = load_images_from_filenames(X_train_filenames, image_dir, img_size)
X_val_images = load_images_from_filenames(X_val_filenames, image_dir, img_size)
X_test_images = load_images_from_filenames(X_test_filenames, image_dir, img_size)

# Normalize image data
X_train_images = X_train_images / 255.0
X_val_images = X_val_images / 255.0
X_test_images = X_test_images / 255.0

# Check shapes of the datasets
X_train_images.shape, X_val_images.shape, X_test_images.shape

((2672, 224, 224, 3), (573, 224, 224, 3), (573, 224, 224, 3))

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.data import AUTOTUNE
from tensorflow.keras.applications import ResNet50


# Limit TensorFlow to use only 2 CPU threads
import os
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["TF_NUM_INTRAOP_THREADS"] = "2"
os.environ["TF_NUM_INTEROP_THREADS"] = "2"

# Preprocess function
def preprocess(image, label):
    image = tf.image.resize(image, [128, 128]) / 255.0  # Resize and normalize
    return image, label

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_images, y_train_labels))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(64).prefetch(AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val_images, y_val_labels))
val_dataset = val_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).batch(64).prefetch(AUTOTUNE)

# Define model
base_model = ResNet50(weights=None, include_top=False, input_shape=(128, 128, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_mapping), activation='softmax')  # Replace `len(label_mapping)` with number of classes
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1355)              348235    
                                                                 
Total params: 24,460,491
Trainable params: 24,407,371
Non-trainable params: 53,120
_______________________________________

In [ ]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/10
42/42 [==============================] - 373s 9s/step - loss: 5.7510 - accuracy: 0.3308 - val_loss: 6.6850 - val_accuracy: 0.3543
Epoch 2/10
42/42 [==============================] - 359s 9s/step - loss: 5.2148 - accuracy: 0.3668 - val_loss: 6.2742 - val_accuracy: 0.3543
Epoch 3/10
42/42 [==============================] - 361s 9s/step - loss: 5.0633 - accuracy: 0.3664 - val_loss: 6.0200 - val_accuracy: 0.3543
Epoch 4/10
42/42 [==============================] - 358s 9s/step - loss: 4.9714 - accuracy: 0.3656 - val_loss: 5.7804 - val_accuracy: 0.3543
Epoch 5/10
42/42 [==============================] - 358s 9s/step - loss: 4.9124 - accuracy: 0.3656 - val_loss: 5.7190 - val_accuracy: 0.3543
Epoch 6/10
42/42 [==============================] - 358s 9s/step - loss: 4.8447 - accuracy: 0.3664 - val_loss: 5.6186 - val_accuracy: 0.3543
Epoch 7/10
42/42 [==============================] - 359s 9s/step - loss: 4.7770 - accuracy: 0.3660 - val_loss: 5.7118 - val_accuracy: 0.3543
Epoch 8/10
42

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# 실제 테스트 데이터에 사용된 클래스 확인
used_classes = np.unique(y_test_labels)

# 성능 지표 계산
accuracy = accuracy_score(y_test_labels, y_pred_labels)
precision = precision_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test_labels, y_pred_labels, average='weighted', zero_division=0)

# 분류 보고서 생성
# label_encoder가 있는 경우에만 target_names 설정
if "label_encoder" in globals():
    target_names = [label_encoder.classes_[i] for i in used_classes]
else:
    target_names = [str(i) for i in used_classes]

report = classification_report(
    y_test_labels,
    y_pred_labels,
    labels=used_classes,
    target_names=target_names,
    zero_division=0
)

# 결과 출력
print("=== 성능 지표 ===")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-Score  : {f1:.4f}")
